<a href="https://colab.research.google.com/github/GeoExploreTech/APP_BACKEND_NODEJS/blob/master/IFC_2_CITYJSON_GEOM_SETTING_TOPO_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and set up Conda in Google Colab using condacolab

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


Here's a summarized comment for your code:  

```python
# Create a Conda environment with Python 3.10, activate it, and install required packages:
# - pythonocc-core (version 7.8.1.1) from conda-forge
# - pandas, lark, ifcopenshell, and pythreejs
!conda create --name=pyoccenv python=3.10
!source activate pyoccenv
!conda install -c conda-forge pythonocc-core=7.8.1.1
!conda install pandas lark ifcopenshell pythreejs
```

### Note:
- The `!source activate pyoccenv` command might need adjustment in some systems or scripts (e.g., using `conda activate pyoccenv` depending on the shell).

In [ ]:
!conda create --name=pyoccenv python=3.10
!source activate pyoccenv
!conda install -c conda-forge pythonocc-core=7.8.1.1
!conda install pandas lark ifcopenshell pythreejs

In [2]:
# Import necessary library
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.element
import ifcopenshell.util.shape
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer, format_color

from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Solid, TopoDS_Shell, TopoDS_Face, topods
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.BRepCheck import BRepCheck_Analyzer
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopAbs import TopAbs_VERTEX, TopAbs_FACE,TopAbs_ShapeEnum
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh


import uuid
import json
import multiprocessing
import numpy as np
from scipy.optimize import least_squares

import shapely
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union
from shapely.validation import make_valid

import math
import re


This function, get_ifc_model, is a simple utility to load an IFC model using the Ifcopenshell library. Here's a concise explanation as a comment:

In [3]:
# IFC file uploaded to IFC_Model
def get_ifc_model(ifc_file_name):
    """Get the IFC model from the uploaded file."""
    return ifcopenshell.open(ifc_file_name)  # Open the IFC file

Here's a concise explanation of the `extract_a_space_semantics` function as a comment:

```python
def extract_a_space_semantics(element):
    """
    Extract semantic information from a specific IfcSpace element.

    Returns a dictionary with:
    - "id": Global ID of the element.
    - "type": IFC type of the element.
    - "name": Name of the element (defaults to "Unnamed" if not set).
    - "attributes": Dictionary of property names and their values.
    """
    return {
        "id": element.GlobalId,
        "type": element.is_a(),
        "name": element.Name if element.Name else "Unnamed",
        "attributes": {
            prop.Name: prop.NominalValue.wrappedValue if hasattr(prop.NominalValue, 'wrappedValue') else prop.NominalValue
            for prop in element.IsDefinedBy[0].RelatingPropertyDefinition.HasProperties
            if hasattr(element, 'IsDefinedBy') and element.IsDefinedBy
        }
    }
```
### Key Points:
1. **Input:** The function takes an IFC element (e.g., `IfcSpace`) as input.
2. **Output:** A dictionary containing:
   - **Global ID:** A unique identifier for the element.
   - **IFC Type:** The type of the element (e.g., `IfcSpace`).
   - **Name:** The name of the element, with "Unnamed" as a fallback.
   - **Attributes:** A dictionary of property names and values from the element's associated property definitions.  
3. **Error Handling:** The function uses checks (`hasattr`) to ensure it only accesses valid attributes, avoiding errors on missing properties.

In [4]:
# Function for Extracting Semantics Model from Space
def extract_a_space_semantics(element):
    """Extract semantics from the IFC from a particular IfcSpace element"""

    return {
        "id": element.GlobalId,
        "type": element.is_a(),
        "name": element.Name if element.Name else "Unnamed",
        "attributes": {prop.Name: prop.NominalValue.wrappedValue if hasattr(prop.NominalValue, 'wrappedValue') else prop.NominalValue
                        for prop in element.IsDefinedBy[0].RelatingPropertyDefinition.HasProperties
                        if hasattr(element, 'IsDefinedBy') and element.IsDefinedBy}
    }


In [6]:
def extract_string_value(data):
  """
  Extracts the string value enclosed within IfcIdentifier, IfcLabel, or IfcText
  from a given dictionary.

  Args:
    data: The dictionary object containing the data.

  Returns:
    A dictionary containing the extracted string values.
  """

  extracted_values = {}
  pattern = r"Ifc(Identifier|Label|Text)\('(.*?)'\)"

  for key, value in data.items():
    if value:
      # Convert value to string before applying regex
      try:
        value_str = str(value)
      except:
        value_str = "" # Handle cases where conversion to string might fail

      match = re.search(pattern, value_str)
      if match:
        extracted_values[key] = match.group(2)

  return extracted_values

In [7]:
# Function for Extracting Zones Attribute from IFC file
def get_ifc_zone_attributes_per_one(zone):
    """
    Extracts and prints all semantic attributes and custom properties from IfcZone entities in an IFC file.

    Args:
        zone (element): IfcZone element.

    Returns:
        dict: A dictionary where keys are IfcZone names and values are dictionaries of attributes and custom properties.
    """

    zone_data = {}

    # Extract standard attributes
    for attr_name in dir(zone):
        if not attr_name.startswith("__") and not callable(getattr(zone, attr_name)):
            zone_data[attr_name] = getattr(zone, attr_name)

    # Extract custom properties
    custom_properties = {}
    for rel in zone.IsDefinedBy:
        if rel.is_a("IfcRelDefinesByProperties"):
            prop_set = rel.RelatingPropertyDefinition
            if prop_set.is_a("IfcPropertySet"):
                for prop in prop_set.HasProperties:
                    if prop.Name != "Type":
                        custom_properties[prop.Name] = getattr(prop, "NominalValue", None)


    # Add custom properties to the data
    zone_data["CustomProperties"] = extract_string_value(custom_properties)

    return zone_data

In [ ]:
# Check if the shape is valid
def is_valid_shape(shape):
    analyzer = BRepCheck_Analyzer(shape)
    return analyzer.IsValid()

In [ ]:
# Get faces from TopoDS_Compound or TopoDS_Shell
def get_faces(shape):
    faces = []
    if isinstance(shape, TopoDS_Compound):
        explorer = TopExp_Explorer(shape, TopAbs_FACE) # Use TopAbs_FACE instead of TopoDS_Face
        while explorer.More():
            face = explorer.Current()
            faces.append(face)
            explorer.Next()
    elif isinstance(shape, (TopoDS_Solid, TopoDS_Shell)):
        explorer = TopExp_Explorer(shape, TopAbs_FACE) # Use TopAbs_FACE instead of TopoDS_Face
        while explorer.More():
            face = explorer.Current()
            faces.append(face)
            explorer.Next()
    elif isinstance(shape, TopoDS_Face):
        faces.append(shape)
    return faces

In [ ]:
def get_vertices(face):
    vertices = []
    explorer = TopExp_Explorer(face, TopAbs_VERTEX)
    while explorer.More():
        vertex = explorer.Current()
        point = BRep_Tool.Pnt(vertex)
        vertices.append((point.X(), point.Y(), point.Z()))
        explorer.Next()
    return vertices

In [ ]:
def create_cityjson_with_attribute_data(ifc_file_model:ifcopenshell):

    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.USE_PYTHON_OPENCASCADE, True)


    cityjson = {
        "type": "CityJSON",
        "version": "2.0",  # Or your desired version
        "CityObjects": {},
        "vertices": [],
        "transform": {
            "scale": [1.0, 1.0, 1.0], # Default scaling
            "translate": [0.0, 0.0, 0.0] # Default translation
        },
      "metadata": {
        "referenceSystem": ""
      },
    }

    vertex_index = 0

    # Retrieve all IfcZone entities
    ifc_zones = ifc_file_model.by_type("IfcZone")

    # Iterate through all zone element
    for zone in ifc_zones:

        zone_data = get_ifc_zone_attributes_per_one(zone)
        zone_info = {
            "ZoneName": zone_data["Name"],
            "ZoneDetails":{**zone_data["CustomProperties"]}
            }


        for rel in zone.IsGroupedBy:
            if rel.is_a("IfcRelAssignsToGroup"):
                for space in rel.RelatedObjects:
                    if space.is_a("IfcSpace"):
                        # print(space.Name)
                        space_info = extract_a_space_semantics(space)
                        space_detail = {**space_info["attributes"]}
                        space_attributes = {
                            "SpaceName": space_info["name"],
                            }
                        # print(space_info)
                        if space.Representation is not None:
                            shape = ifcopenshell.geom.create_shape(settings, space)
                            geometry = shape.geometry
                            if not is_valid_shape(geometry):
                                print("Invalid geometry. Skipping...")
                            else:
                                # print(geometry)
                                faces = get_faces(geometry)

                            if faces:
                                space_id = space.GlobalId
                                # cityjson["CityObjects"][zone.GlobalId]["members"].append(space_id)
                                cityjson["CityObjects"][space_id] = {
                                    "type": "Building",
                                    "attributes": {**zone_info, **space_attributes,**space_detail},
                                    "geometry": [{
                                        "type": "Solid",
                                        "lod": "2",
                                        "boundaries": []
                                    }]
                                }

                                boundaries = []
                                for face in faces:
                                    face_vertices = get_vertices(face)
                                    boundary = []
                                    for vertex in face_vertices:
                                        if vertex not in cityjson["vertices"]:
                                            cityjson["vertices"].append(vertex)
                                        vertex_index = cityjson["vertices"].index(vertex)
                                        boundary.append(vertex_index)
                                    boundaries.append([boundary]) # Double list for CityJSON
                                cityjson["CityObjects"][space_id]["geometry"][0]["boundaries"].append(boundaries)

    return cityjson


In [ ]:
def extract_city_bounding_box_points(cityjson_file):
    """
    Extract the bounding box of all city objects combined as a list of points.

    Args:
        cityjson_file (str): Path to the CityJSON file.

    Returns:
        list: A list of (x, y, z) tuples representing the bounding box points.
    """
    with open(cityjson_file, 'r') as file:
        data = json.load(file)

    # Fetch vertices from CityJSON file
    vertices = data.get("vertices", [])

    if not vertices:
        raise ValueError("No vertices found in the CityJSON file.")

    # Calculate bounding box
    min_x = min(v[0] for v in vertices)
    min_y = min(v[1] for v in vertices)
    min_z = min(v[2] for v in vertices)

    max_x = max(v[0] for v in vertices)
    max_y = max(v[1] for v in vertices)
    max_z = max(v[2] for v in vertices)

    # Create a list of points representing the 3D bounding box
    bounding_box_points = [
        (min_x, min_y, min_z),
        (min_x, max_y, min_z),
        (max_x, max_y, min_z),
        (max_x, min_y, min_z),
        (min_x, min_y, min_z),  # Close the polygon
        (min_x, min_y, max_z),
        (min_x, max_y, max_z),
        (max_x, max_y, max_z),
        (max_x, min_y, max_z),
        (min_x, min_y, max_z)  # Close the upper polygon
    ]

    return bounding_box_points




In [ ]:
import numpy as np

np.random.seed(0)  # for reproducibility

def compute_transformation_params(local_coords, global_coords):
    """
    Computes transformation parameters (scale, rotation, translation)
    for converting from a local coordinate system to a global coordinate system.

    Args:
        local_coords (numpy.ndarray): Array of local coordinates (Nx2).
        global_coords (numpy.ndarray): Array of corresponding global coordinates (Nx2).

    Returns:
        tuple: A tuple containing:
            - scale (float): Scale factor.
            - rotation (float): Rotation angle in radians (counter-clockwise).
            - translation (numpy.ndarray): Translation vector (1x2).
            Returns None if the input data is invalid.
        Raises:
            ValueError: If the input arrays have incompatible shapes or if there are insufficient points.
    """

    local_coords = np.asarray(local_coords)
    global_coords = np.asarray(global_coords)

    if local_coords.shape != global_coords.shape or local_coords.shape[1] != 2:
        raise ValueError("Input coordinate arrays must have the same shape (Nx2).")

    n_points = local_coords.shape[0]
    if n_points < 2:  # Need at least 2 points for affine transformation
        raise ValueError("At least two corresponding points are required for transformation.")

    # Calculate centroids
    local_centroid = np.mean(local_coords, axis=0)
    global_centroid = np.mean(global_coords, axis=0)

    # Center the coordinates
    local_centered = local_coords - local_centroid
    global_centered = global_coords - global_centroid

    # Calculate the covariance matrix
    covariance_matrix = np.dot(global_centered.T, local_centered)

    # Calculate the singular value decomposition (SVD)
    U, S, V = np.linalg.svd(covariance_matrix)

    # Calculate the rotation matrix
    rotation_matrix = np.dot(U, V)

    # Handle reflection case (if det(rotation_matrix) < 0)
    if np.linalg.det(rotation_matrix) < 0:
        V[1, :] *= -1  # Flip the sign of the last row of V
        rotation_matrix = np.dot(U, V)

    # Calculate the rotation angle (in radians)
    rotation = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])

    # Calculate the scale factor
    scale = np.mean(np.linalg.norm(global_centered, axis=1) / np.linalg.norm(local_centered, axis=1))

    # Calculate the translation vector
    translation = global_centroid - scale * np.dot(local_centroid, rotation_matrix)

    return scale, rotation, translation


def test_transformation_accuracy(local_coords, global_coords, tolerance=1e-6):
    """
    Tests the accuracy of the transformation by comparing the transformed local coordinates
    to the original global coordinates.

    Args:
        local_coords (numpy.ndarray): Array of local coordinates (Nx2).
        global_coords (numpy.ndarray): Array of corresponding global coordinates (Nx2).
        tolerance (float): Tolerance for comparing coordinates (default: 1e-6).

    Returns:
        tuple: A tuple containing:
            - rmse (float): Root Mean Square Error of the transformation.
            - max_error (float): Maximum absolute error of the transformation.
            - success (bool): True if all transformed coordinates are within the tolerance, False otherwise.
        Raises:
            ValueError: If the input arrays have incompatible shapes.
    """

    local_coords = np.asarray(local_coords)
    global_coords = np.asarray(global_coords)

    if local_coords.shape != global_coords.shape or local_coords.shape[1] != 2:
        raise ValueError("Input coordinate arrays must have the same shape (Nx2).")

    try:
        scale, rotation, translation = compute_transformation_params(local_coords, global_coords)
    except ValueError as e:
        print(f"Error during transformation parameter computation: {e}")
        return float('inf'), float('inf'), False  # Return infinite error if transformation fails

    # Apply the transformation
    rotation_matrix = np.array([[np.cos(rotation), -np.sin(rotation)], [np.sin(rotation), np.cos(rotation)]])
    transformed_coords = scale * np.dot(local_coords, rotation_matrix) + translation

    # Calculate errors
    errors = np.linalg.norm(transformed_coords - global_coords, axis=1)
    rmse = np.sqrt(np.mean(errors**2))
    max_error = np.max(errors)
    success = np.all(errors <= tolerance)

    return rmse, max_error, success

import math

def compute_transformation_params_least_squares(local_coords, global_coords):
    """
    Computes transformation parameters (scale, rotation, translation) using least squares adjustment.

    Args:
        local_coords (numpy.ndarray): Array of local coordinates (Nx2).
        global_coords (numpy.ndarray): Array of corresponding global coordinates (Nx2).

    Returns:
        tuple: A tuple containing:
            - scale (float): Scale factor.
            - rotation (float): Rotation angle in radians.
            - translation (numpy.ndarray): Translation vector (1x2).
            Returns None if the input data is invalid.
        Raises:
            ValueError: If the input arrays have incompatible shapes or if there are insufficient points.
    """

    local_coords = np.asarray(local_coords)
    global_coords = np.asarray(global_coords)

    if local_coords.shape != global_coords.shape or local_coords.shape[1] != 2:
        raise ValueError("Input coordinate arrays must have the same shape (Nx2).")

    n_points = local_coords.shape[0]
    if n_points < 2:  # Need at least 2 points for affine transformation
        raise ValueError("At least two corresponding points are required for transformation.")

    # Construct the design matrix A and the observation vector l
    A = np.zeros((2 * n_points, 4))
    l = global_coords.flatten()

    for i in range(n_points):
        A[2 * i, 0] = local_coords[i, 0]
        A[2 * i, 1] = -local_coords[i, 1]
        A[2 * i, 2] = 1
        A[2 * i + 1, 0] = local_coords[i, 1]
        A[2 * i + 1, 1] = local_coords[i, 0]
        A[2 * i + 1, 3] = 1

    # Least squares solution: x = (A^T * A)^-1 * A^T * l
    try:
        x = np.linalg.solve(A.T @ A, A.T @ l)
    except np.linalg.LinAlgError:
        print("Singular matrix encountered. Least squares solution could not be computed.")
        return None

    # Extract the transformation parameters
    scale = np.sqrt(x[0]**2 + x[1]**2)
    rotation = np.arctan2(x[1], x[0])
    translation = np.array([x[2], x[3]])

    return scale, rotation, translation

def transform_coordinates(local_coords, scale, rotation, translation):
    """
    Transforms local coordinates to global coordinates using given transformation parameters.

    Args:
        local_coords (numpy.ndarray): Array of local coordinates (Nx2).
        scale (float): Scale factor.
        rotation (float): Rotation angle in radians.
        translation (numpy.ndarray or list): Translation vector (1x2).

    Returns:
        numpy.ndarray: Array of transformed (global) coordinates (Nx2).
        Raises:
            ValueError: If the input arrays have incompatible shapes.
    """
    local_coords = np.asarray(local_coords)
    translation = np.asarray(translation)

    if local_coords.ndim != 2 or local_coords.shape[1] != 2:
      raise ValueError("Local coordinates must be a Nx2 array.")
    if translation.shape != (2,):
      raise ValueError("Translation must be a 1x2 array or list.")

    rotation_matrix = np.array([[np.cos(rotation), -np.sin(rotation)],
                                [np.sin(rotation), np.cos(rotation)]])

    transformed_coords = scale * np.dot(local_coords, rotation_matrix) + translation

    return transformed_coords


In [ ]:
def transform_cityjson(file_path, output_path, scale, rotation, translation, epsg):
    """
    Transform the vertices of a CityJSON file using Helmert 7-Parameter Transformation.

    Args:
        file_path: Path to the input CityJSON file.
        output_path: Path to save the transformed CityJSON file.
        params: Dictionary containing the Helmert parameters.
    """
    # Load CityJSON file
    with open(file_path, 'r') as f:
        cityjson = json.load(f)
    # Adding reference system
    cityjson["metadata"]["referenceSystem"] = f"https://www.opengis.net/def/crs/EPSG/0/{epsg}"

    # Transform all vertices
    for i, vertex in enumerate(cityjson['vertices']):
        X, Y, Z = vertex
        local_coords_test = np.array([[X, Y]])
        transformed_coords = transform_coordinates(local_coords_test, scale, rotation, translation)[0]
        # X_prime, Y_prime, Z_prime = helmert_transformation(X, Y, Z, params)
        cityjson['vertices'][i] = [transformed_coords[0], transformed_coords[1], Z]

    # Save transformed CityJSON file
    with open(output_path, 'w') as f:
        json.dump(cityjson, f, indent=2)
    print(f"Transformed CityJSON saved to: {output_path}")


In [ ]:
# Step 1: Load the IFC file
ifc_file_name = "/content/bally-ifc.ifc"  # Get the name of the uploade
ifcModel = get_ifc_model(ifc_file_name)

cityjson_data = create_cityjson_with_attribute_data(ifcModel)

# print(cityjson_data)
with open("output_no_georefrence.json", "w") as outfile:
    json.dump(cityjson_data, outfile, indent=4)

print("CityJSON file created successfully!")
# json_result = json.dumps(semanticData, indent=4, default=str)

CityJSON file created successfully!


In [ ]:
# Example usage
boundary = extract_city_bounding_box_points("/content/output_no_georefrence.json")
json_result = json.dumps(boundary, indent=4, default=str)
print(json_result)

In [ ]:
# Computing the coordinate transformation parameters
local_coords = np.array([
    [0.115,-3.20944193],
    [0.11500000,11.90803589],
    [59.42595128,11.90803589],
    [59.42595128,-3.20944193]
])
local_coords_test = np.array([
    [0.115,5]

])
global_coords = np.array([
    [333337.4405,999661.4498],
    [333324.3872,999653.8241],
    [333294.4690,999705.0363],
    [333307.5222,999712.6620]

])


scale, rotation, translation = compute_transformation_params_least_squares(local_coords, global_coords)

print("Scale:", scale)
print("Rotation (radians):", rotation)
print("Translation:", translation)

#     # Test the transformation
#     transformed_coords = scale * np.dot(local_coords, np.array([[np.cos(rotation), -np.sin(rotation)], [np.sin(rotation), np.cos(rotation)]])) + translation
#     print("Transformed coordinates:\n", transformed_coords)
#     print("Original global coordinates:\n", global_coords)

#     # Test accuracy
#     rmse, max_error, success = test_transformation_accuracy(local_coords, global_coords)
#     print("RMSE:", rmse)
#     print("Maximum Error:", max_error)
#     print("Success:", success)

#     transformed_coords2 = transform_coordinates(local_coords_test, scale, rotation, translation)
#     print("Transformed coordinates 2:\n", transformed_coords2[0])

# except ValueError as e:
#     print(f"Error: {e}")


Scale: 1.0000003564278797
Rotation (radians): 2.0995182646239856
Translation: [333334.72727011 999659.73156771]


In [ ]:
# Input and output file paths
input_file = "/content/output_no_georefrence.json"
output_file = "citymodel_transformed.json"

# Transform the CityJSON file
transform_cityjson(input_file, output_file, scale, rotation, translation, 32632)

Transformed CityJSON saved to: citymodel_transformed.json
